In [2]:
from langchain_community.tools import arxiv

In [3]:
from  langchain_community.llms.ollama import Ollama
llm  = Ollama(model="llama3")

In [4]:
from langchain_community.embeddings.ollama import OllamaEmbeddings

In [5]:
from langchain_community.vectorstores import FAISS

In [6]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

Wiki_API_wrapper = WikipediaAPIWrapper(top_k_results=1 , doc_content_chars_max=500)
wiki_query_run = WikipediaQueryRun(api_wrapper=Wiki_API_wrapper)
wiki_query_run

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\PMLS\\Desktop\\ECO_SYSTEM_LANGCHAIN\\myenv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=500))

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI , OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:

import os 
from dotenv import load_dotenv
load_dotenv()
## extracting the api_key
os.environ['OPENAI_API'] = os.getenv("openai_key")

In [27]:
len(os.environ['OPENAI_API_KEY'])

132

In [8]:
llm = Ollama(model='llama3')

In [9]:
context = WebBaseLoader("https://docs.smith.langchain.com")
web_context = context.load()

In [10]:
chunks = RecursiveCharacterTextSplitter(chunk_size = 1000 , chunk_overlap = 500)
split = chunks.split_documents(web_context[:50])
split

[Document(metadata={'source': 'https://docs.smith.langchain.com', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='Get started with LangSmith | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content="Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingS

In [11]:
from langchain.vectorstores import Chroma
vector_ = Chroma.from_documents(split ,embedding= OllamaEmbeddings())

In [12]:
import pickle
pickle.dump(vector_, open('vector_.pkl','wb'))

TypeError: cannot pickle 'sqlite3.Connection' object

# this our own custom created tools 

In [10]:
from langchain.tools.retriever import create_retriever_tool
retrieve_tool = create_retriever_tool(retrieve, "quickstart", "search for LangSmith and LangServe")

In [11]:
retrieve_tool.name

'quickstart'

In [12]:
from langchain_community.tools import ArxivQueryRun
from  langchain_community.utilities import ArxivAPIWrapper
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1 , doc_content_chars_max=500)

arxiv_query_run = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_query_run.name

'arxiv'

In [13]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

In [15]:
llm = ChatOpenAI(api_key=os.environ['OPENAI_API'] , model='gpt-3.5-turbo')

In [16]:
## Creating the agents for working flow of the tools 
tools = [wiki_query_run,arxiv_query_run,retrieve_tool]
from langchain.agents import AgentExecutor
from langchain.agents import create_openai_tools_agent
agents = create_openai_tools_agent(llm,tools,prompt)

In [17]:
agents_executor = AgentExecutor(agent=agents, tools=tools,verbose=True)

In [20]:
agents_executor.invoke({'input' : "what is Log your first trace"})



> Entering new AgentExecutor chain...

Invoking: `quickstart` with `{'query': 'Log your first trace'}`


pipeline("Hello, world!")// Out: Hello there! How can I assist you today?View a sample output trace.Learn more about tracing in the how-to guides.5. Run your first evaluation​Evaluation requires a system to test, data to serve as test cases, and optionally evaluators to grade the results. Here we use a built-in accuracy evaluator.PythonTypeScriptfrom langsmith import Clientfrom langsmith.evaluation import evaluateclient = Client()# Define dataset: these are your test casesdataset_name = "Sample Dataset"dataset = client.create_dataset(dataset_name, description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evalu

{'input': 'what is Log your first trace',
 'output': '"Log your first trace" is related to logging traces in LangSmith, a platform for building production-grade LLM applications. It involves using the `traceable` function to log traces to LangSmith, allowing for close monitoring and evaluation of your application. The process includes steps such as installing LangSmith, creating an API key, setting up the environment, and using the `traceable` function to log traces. If you need more detailed information or assistance with this, feel free to ask!'}

In [19]:
agents_executor.invoke({'input' : "what is Quantum computer "})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Quantum computer'}`


Page: Quantum computing
Summary: A quantum computer is a computer that exploits quantum mechanical phenomena. On small scales, physical matter exhibits properties of both particles and waves, and quantum computing leverages this behavior using specialized hardware. Classical physics cannot explain the operation of these quantum devices, and a scalable quantum computer could perform some calculations exponentially faster than any modern "classical" computer. In particular, a large-scale quantum cA quantum computer is a computer that exploits quantum mechanical phenomena. It leverages the properties of particles and waves on small scales to perform calculations exponentially faster than classical computers. Quantum computing uses specialized hardware that classical physics cannot explain.

> Finished chain.


{'input': 'what is Quantum computer ',
 'output': 'A quantum computer is a computer that exploits quantum mechanical phenomena. It leverages the properties of particles and waves on small scales to perform calculations exponentially faster than classical computers. Quantum computing uses specialized hardware that classical physics cannot explain.'}

In [21]:
def get_embedding(text, model="model-identifier"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

print(get_embedding("Once upon a time, there was a cat."))